In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
os.chdir('/home1/10031/gracesstew/ACSC_SC25')

In [3]:
###import enterococcus dataset
magic = pd.read_csv('data/magic_island_bowls.csv')

In [4]:
###sort columns
columns = ["collection date",
           #"collection time (Pacific/Honolulu)", 
           #"water temperature (f)",
           "Enterococcus (mpn/100mL)"]

In [5]:
##### MAGIC ISLAND #####
#print(alas.columns)
new_magic = magic[columns]
print(new_magic.columns)

Index(['collection date', 'Enterococcus (mpn/100mL)'], dtype='object')


In [6]:
new_magic = new_magic.dropna()
new_magic["collection date"].min()
###  1/10/2021

'1/10/2021'

In [7]:
###import tourism data

tourism = pd.read_csv('data/tourism.csv')
display(tourism)

,Indicator,Market,Destination,Units,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,...,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
0,Visitor arrivals,Air total,Oahu,persons,"454,308","503,033","445,058","482,224","536,414","561,784",...,"433,889","450,281","444,095","533,420","457,278","431,349","525,982","487,707","466,401","512,198"
1,Data is updated monthly by the Research & Econ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Source of Data: Hawaii Tourism Authority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Seasonally adjusted series are from DBEDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hotel performance data prior to March 2017 are...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# drop rows 1-4
new_tour = tourism.drop(tourism.index[1:5])
display(new_tour)

,Indicator,Market,Destination,Units,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,...,2024-09,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06
0,Visitor arrivals,Air total,Oahu,persons,"454,308","503,033","445,058","482,224","536,414","561,784",...,"433,889","450,281","444,095","533,420","457,278","431,349","525,982","487,707","466,401","512,198"


In [9]:
tour_transposed = new_tour.transpose()
tour_transposed = tour_transposed.reset_index()
tour_transposed = tour_transposed.rename(columns={'index': 'date'})
tour_transposed = tour_transposed.drop(tour_transposed.index[0:4])

tour_transposed.columns.values[1] = "visitors"
tour_transposed.head()

,date,visitors
4,2018-02,"454,308"
5,2018-03,"503,033"
6,2018-04,"445,058"
7,2018-05,"482,224"
8,2018-06,"536,414"


In [10]:
##save to csv
#tour_transposed.to_csv("new_tourism.csv", index=False)

##load csvs
tourism = pd.read_csv('data/new_tourism.csv')
#display(tourism)

yacht = pd.read_csv('/work2/10031/gracesstew/PACIOOS/PacIOOS_yacht.csv')
#display(yacht)

In [11]:
new_magic['collection date'] = (
    pd.to_datetime(new_magic['collection date'])
      .dt.strftime('%Y-%m')
)

magic_avg = (
    new_magic.groupby('collection date')['Enterococcus (mpn/100mL)']
      .mean()
      .reset_index()
)

magic_avg = magic_avg.rename(columns={"collection date": "year_month"})
magic_avg.head()

,year_month,Enterococcus (mpn/100mL)
0,2019-01,10.0
1,2019-02,10.0
2,2019-03,10.0
3,2019-04,20.5
4,2019-05,10.0


In [12]:
tourism  = tourism.rename(columns={"date": "year_month"})
tourism  = tourism.rename(columns={"visitors": "Visitors"})
tourism.head()

,year_month,Visitors
0,2018-02,"454,308"
1,2018-03,"503,033"
2,2018-04,"445,058"
3,2018-05,"482,224"
4,2018-06,"536,414"


In [13]:
yacht  = yacht.rename(columns={"month": "year_month"})
yacht.head()

,year_month,Water Temperature (degF),Salinity (PSU),Turbidity (NTU),Chlorophyll (ug/L)
0,2018-01,79.891204,31.723171,1.919315,3.220971
1,2018-02,79.309341,31.119680,1.639482,2.639715
2,2018-03,79.152026,31.250279,1.481051,2.529561
3,2018-04,80.045995,31.349549,3.003634,2.544006
4,2018-05,79.881787,32.353888,1.395863,4.219175


In [16]:
for df in [yacht, tourism, magic_avg]:
    df["year_month"] = pd.to_datetime(df["year_month"]).dt.to_period("M").astype(str)

from functools import reduce

tourist = [tourism, yacht, magic_avg]
tour_merge = reduce(lambda left, right: left.merge(right, on="year_month", how="outer"), tourist)

tour_merge.head()

#save to csv
#tour_merge.to_csv('all_tourist.csv', index=False)

,year_month,Visitors,Water Temperature (degF),Salinity (PSU),Turbidity (NTU),Chlorophyll (ug/L),Enterococcus (mpn/100mL)
0,2018-01,NaN,79.891204,31.723171,1.919315,3.220971,NaN
1,2018-02,"454,308",79.309341,31.119680,1.639482,2.639715,NaN
2,2018-03,"503,033",79.152026,31.250279,1.481051,2.529561,NaN
3,2018-04,"445,058",80.045995,31.349549,3.003634,2.544006,NaN
4,2018-05,"482,224",79.881787,32.353888,1.395863,4.219175,NaN
